In [78]:
import pandas as pd
import numpy as np

In [79]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])


https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [80]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [81]:
from sklearn.model_selection import train_test_split
X = data.drop(['MEDV'], axis=1)
y = data['MEDV']

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = RANDOM_STATE)

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [83]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
reg = LinearRegression()
rid = Ridge()
las = Lasso()
LR = reg.fit(X_train, y_train)
RD = rid.fit(X_train, y_train)
LS = las.fit(X_train, y_train)
lreg_pr = reg.predict(X_test)
rid_pr = rid.predict(X_test)
las_pr = las.predict(X_test)
r2_lr = r2_score(y_test, lreg_pr)
r2_ridge = r2_score(y_test, rid_pr)
r2_lasso = r2_score(y_test, las_pr)
results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [84]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
intervals = [10**-5,10**-4,10**-3,10**-2,10**-1,10**0,10**1,10**2,10**3,10**4,10**5]

R_GS_CV = GridSearchCV(rid, {'alpha':intervals})
L_GS_CV = GridSearchCV(las, {'alpha':intervals})
R_CV = R_GS_CV.fit(X_train, y_train)
L_CV = L_GS_CV.fit(X_train, y_train)

print(R_CV.best_params_)
print(L_CV.best_params_)

R1 = R_CV.predict(X_test)
L1 = L_CV.predict(X_test)

R_R_CV = RidgeCV(alphas=intervals)
L_L_CV = LassoCV(alphas = intervals)
RCV = R_R_CV.fit(X_train, y_train)
LCV = L_L_CV.fit(X_train, y_train)
R2 = RCV.predict(X_test)
L2 = LCV.predict(X_test)

r2_ridge_grid_search = r2_score(y_test, R1)
r2_ridge_cv = r2_score(y_test, R2)
r2_lasso_grid_search = r2_score(y_test, L1)
r2_lasso_cv = r2_score(y_test, L2)
results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]
results_regression

{'alpha': 1e-05}
{'alpha': 1e-05}


,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483


После выполнения задания, можно сделать вывод, что результаты полученные с использованием способов из задания одинаковые. Лучший коэффициент регуляризации - 'alpha': 1e-05, результаты почти одинаковые с результатами из предыдущего задания.

4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [85]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

estimators_SS_R = [('StandardScaler', StandardScaler()), ('model', Ridge())]
estimators_SS_L = [('StandardScaler', StandardScaler()), ('model', Lasso())]
estimators_MMS_R = [('MinMaxScaler', MinMaxScaler()), ('model', Ridge())]
estimators_MMS_L = [('MinMaxScaler', MinMaxScaler()), ('model', Lasso())]

P_SS_R = Pipeline(estimators_SS_R).fit(X_train, y_train)
r2_ridge_standart_scaler = r2_score(y_test, P_SS_R.predict(X_test))

P_SS_L = Pipeline(estimators_SS_L).fit(X_train, y_train)
r2_lasso_standart_scaler = r2_score(y_test, P_SS_L.predict(X_test))

P_MMS_R = Pipeline(estimators_MMS_R).fit(X_train, y_train)
r2_ridge_min_max_scaler = r2_score(y_test, P_MMS_R.predict(X_test))

P_MMS_L = Pipeline(estimators_MMS_L).fit(X_train, y_train)
r2_lasso_min_max_scaler = r2_score(y_test, P_MMS_L.predict(X_test))

results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


После выполнения задания, можно сделать вывод, что масштабирование признаков практически не повлияло на качество регрессии

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [86]:
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV

estimators_SS_R_alph = [('StandardScaler', StandardScaler()), ('model', RidgeCV(alphas=intervals))]
estimators_SS_L_alph = [('StandardScaler', StandardScaler()), ('model', LassoCV(alphas=intervals))]
estimators_MMS_R_alph = [('MinMaxScaler', MinMaxScaler()), ('model', RidgeCV(alphas=intervals))]
estimators_MMS_L_alph = [('MinMaxScaler', MinMaxScaler()), ('model', LassoCV(alphas=intervals))]

P_SS_R_a = Pipeline(estimators_SS_R_alph).fit(X_train, y_train)
r2_ridge_standart_scaler_cv = r2_score(y_test, P_SS_R_a.predict(X_test))

P_SS_L_a = Pipeline(estimators_SS_L_alph).fit(X_train, y_train)
r2_lasso_standart_scaler_cv = r2_score(y_test, P_SS_L_a.predict(X_test))

P_MMS_R_a = Pipeline(estimators_MMS_R_alph).fit(X_train, y_train)
r2_ridge_min_max_scaler_cv = r2_score(y_test, P_MMS_R_a.predict(X_test))

P_MMS_L_a = Pipeline(estimators_MMS_L_alph).fit(X_train, y_train)
r2_lasso_min_max_scaler_cv = r2_score(y_test, P_MMS_L_a.predict(X_test))

results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


После выполнения задания, можно сделать вывод, что использование регуляризации и масштабирования признаков дает такие же результаты как простые Ridge и Lasso

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [87]:
from sklearn.preprocessing import PolynomialFeatures

estimators_SS_R_PF = [('StandardScaler', StandardScaler()), ('transform', PolynomialFeatures()), ('model', Ridge())]
estimators_SS_L_PF = [('StandardScaler', StandardScaler()), ('transform', PolynomialFeatures()), ('model', Lasso())]
estimators_MMS_R_PF = [('MinMaxScaler', MinMaxScaler()), ('transform', PolynomialFeatures()), ('model', Ridge())]
estimators_MMS_L_PF = [('MinMaxScaler', MinMaxScaler()), ('transform', PolynomialFeatures()), ('model', Lasso())]

P_SS_R_PF = Pipeline(estimators_SS_R_PF).fit(X_train, y_train)
r2_ridge_standart_scaler_poly = r2_score(y_test, P_SS_R_PF.predict(X_test))

P_SS_L_PF = Pipeline(estimators_SS_L_PF).fit(X_train, y_train)
r2_lasso_standart_scaler_poly = r2_score(y_test, P_SS_L_PF.predict(X_test))

P_MMS_R_PF = Pipeline(estimators_MMS_R_PF).fit(X_train, y_train)
r2_ridge_min_max_scaler_poly = r2_score(y_test, P_MMS_R_PF.predict(X_test))

P_MMS_L_PF = Pipeline(estimators_MMS_L_PF).fit(X_train, y_train)
r2_lasso_min_max_scaler_poly = r2_score(y_test, P_MMS_L_PF.predict(X_test))


results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


После выполнения задания, можно сделать вывод, что добавление попарных произведений признаков и их квадратов на масштабированных признаках привело к значительному улучшению качества обучения моделей, кроме Lasso_MinMaxScaler

7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [88]:
import warnings
warnings.filterwarnings("ignore")

estimators_SS_R_PF_CV = [('StandardScaler', StandardScaler()), ('transform', PolynomialFeatures()), ('model', RidgeCV(alphas=intervals))]
estimators_SS_L_PF_CV = [('StandardScaler', StandardScaler()), ('transform', PolynomialFeatures()), ('model', LassoCV(alphas=intervals))]
estimators_MMS_R_PF_CV = [('MinMaxScaler', MinMaxScaler()), ('transform', PolynomialFeatures()), ('model', RidgeCV(alphas=intervals))]
estimators_MMS_L_PF_CV = [('MinMaxScaler', MinMaxScaler()), ('transform', PolynomialFeatures()), ('model', LassoCV(alphas=intervals))]

P_SS_R_PF_CV = Pipeline(estimators_SS_R_PF_CV).fit(X_train, y_train)
r2_ridge_standart_scaler_poly_cv = r2_score(y_test, P_SS_R_PF_CV.predict(X_test))

P_SS_L_PF_CV = Pipeline(estimators_SS_L_PF_CV).fit(X_train, y_train)
r2_lasso_standart_scaler_poly_cv = r2_score(y_test, P_SS_L_PF_CV.predict(X_test))

P_MMS_R_PF_CV = Pipeline(estimators_MMS_R_PF_CV).fit(X_train, y_train)
r2_ridge_min_max_scaler_poly_cv = r2_score(y_test, P_MMS_R_PF_CV.predict(X_test))

P_MMS_L_PF_CV = Pipeline(estimators_MMS_L_PF_CV).fit(X_train, y_train)
r2_lasso_min_max_scaler_poly_cv = r2_score(y_test, P_MMS_L_PF_CV.predict(X_test))


results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


После выполнения задания, видно, что качество обучения моделей где-то улучшилось, где-то ухудшилось

8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [89]:
param = dict(normalizer=[StandardScaler(), MinMaxScaler()],
                  polynomial__degree=[2],
                  classifier=[Ridge(), Lasso()],
                  classifier__alpha=intervals)
pipe = Pipeline([('polynomial', PolynomialFeatures()), ('normalizer', StandardScaler()), ('classifier', Ridge())])
GS = GridSearchCV(pipe, param, scoring='r2').fit(X_train, y_train)
best_params = GS.best_params_
print('Параметры лучшей модели:\n', best_params)
r2_best_model = r2_score(y_test, GS.predict(X_test))
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]
results_regression

Параметры лучшей модели:
 {'classifier': Ridge(alpha=0.1), 'classifier__alpha': 0.1, 'normalizer': MinMaxScaler(), 'polynomial__degree': 2}


,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


http://archive.ics.uci.edu/ml/datasets/Adult

In [90]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [91]:
X = data[data.columns[:-1]]
y = data[data.columns[-1:]].applymap(lambda x: 1 if str(x) == "<=50K" else 0)
y

,class
0,1
1,1
2,1
3,1
4,1
...,...
48837,1
48838,1
48839,1
48840,1


10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [92]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
zn = y['class'].value_counts()
print(zn) #видим, что часто всречаемый класс - 1(37155)
X['frequently'] = 1 #добавим новую колонку с самым частовсречающимся значением
y_pred =X['frequently'] 
f1_most_frequent = f1_score(y, y_pred)
acc_most_frequent = accuracy_score(y, y_pred)
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]
results_classification

1    37155
0    11687
Name: class, dtype: int64


,model,task,f1,accuracy
0,Most Frequent class,task10,0.8641,0.760718


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [93]:
data.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64

12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [94]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       48842 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      48842 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48842 non-null  object
 14  class           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [95]:
integ = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
X_int = X[integ]
print(X_int)
categ = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
X_cat = X[categ]
print(X_cat)

       age  fnlwgt  education-num  capital-gain  capital-loss  hours-per-week
0       39   77516             13          2174             0              40
1       50   83311             13             0             0              13
2       38  215646              9             0             0              40
3       53  234721              7             0             0              40
4       28  338409             13             0             0              40
...    ...     ...            ...           ...           ...             ...
48837   39  215419             13             0             0              36
48838   64  321403              9             0             0              40
48839   38  374983             13             0             0              50
48840   44   83891             13          5455             0              40
48841   35  182148             13             0             0              60

[48842 rows x 6 columns]
              workclass  education    

13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [96]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

estimators_MMS = [("MinMaxScaler", MinMaxScaler())]
estimators_OHE = [("OneHotEncoder", OneHotEncoder(handle_unknown="ignore"))]
P_MMS = Pipeline(estimators_MMS)
P_OHE = Pipeline(estimators_OHE)
COL_TR = ColumnTransformer([("categ", P_OHE, categ), ("integ", P_MMS, integ)])
new_df = COL_TR.fit_transform(X)
X_new_df = pd.DataFrame(new_df.toarray())

LOG_reg = LogisticRegression(max_iter=1000)
KNClass = KNeighborsClassifier()
LIN_scv = LinearSVC()

acc_LR = cross_val_score(LOG_reg, X_new_df, y, scoring='accuracy').mean()
f1_LR = cross_val_score(LOG_reg, X_new_df, y, scoring='f1').mean()

acc_KNN = cross_val_score(KNClass, X_new_df, y, scoring='accuracy').mean()
f1_KNN = cross_val_score(KNClass, X_new_df, y, scoring='f1').mean()

f1_SVM = cross_val_score(LIN_scv, X_new_df, y, scoring='f1').mean()
acc_SVM = cross_val_score(LIN_scv, X_new_df, y, scoring='accuracy').mean()


results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]

In [97]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.864100,0.760718
1,LogisticRegression,task13,0.904800,0.850846
2,KNeighborsClassifier,task13,0.886996,0.824782
3,LinearSVC,task13,0.906322,0.852914


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [103]:
from sklearn.impute import SimpleImputer

SI = SimpleImputer(strategy='most_frequent', missing_values='?')
X_mod_df = pd.DataFrame(SI.fit_transform(X), columns=X.columns)

new_df_MOD = COL_TR.fit_transform(X_mod_df)
X_new_df_MOD = pd.DataFrame(new_df_MOD.toarray())

acc_LR = cross_val_score(LOG_reg, X_new_df_MOD, y, scoring='accuracy').mean()
f1_LR = cross_val_score(LOG_reg, X_new_df_MOD, y, scoring='f1').mean()

acc_KNN = cross_val_score(KNClass, X_new_df_MOD, y, scoring='accuracy').mean()
f1_KNN = cross_val_score(KNClass, X_new_df_MOD, y, scoring='f1').mean()

f1_SVM = cross_val_score(LIN_scv, X_new_df_MOD, y, scoring='f1').mean()
acc_SVM = cross_val_score(LIN_scv, X_new_df_MOD, y, scoring='accuracy').mean()

results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]

In [59]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [104]:
index = X[(X=='?').sum(axis=1) > 0].index
X_del = X.drop(index = index)

new_df_DEL = COL_TR.fit_transform(X_del)
X_new_df_DEL = pd.DataFrame(new_df_DEL.toarray())

acc_LR_del_missings = cross_val_score(LOG_reg, X_new_df_DEL, y.drop(index = index), scoring='accuracy').mean()
f1_LR_del_missings = cross_val_score(LOG_reg, X_new_df_DEL, y.drop(index = index), scoring='f1').mean()

acc_KNN_del_missings = cross_val_score(KNClass, X_new_df_DEL, y.drop(index = index), scoring='accuracy').mean()
f1_KNN_del_missings = cross_val_score(KNClass, X_new_df_DEL, y.drop(index = index), scoring='f1').mean()

f1_SVM_del_missings = cross_val_score(LIN_scv, X_new_df_DEL, y.drop(index = index), scoring='f1').mean()
acc_SVM_del_missings = cross_val_score(LIN_scv, X_new_df_DEL, y.drop(index = index), scoring='accuracy').mean()


results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

In [106]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.864100,0.760718
1,LogisticRegression,task13,0.904800,0.850846
2,KNeighborsClassifier,task13,0.886996,0.824782
3,LinearSVC,task13,0.906322,0.852914
12,Best_Model,task17,0.915395,0.866386
4,LogisticRegression_impute,task14,0.904659,0.850477
5,KNeighborsClassifier_impute,task14,0.887164,0.824966
6,LinearSVC_impute,task14,0.905142,0.850825
7,LogisticRegression_delete_missings,task15,0.901152,0.846845
8,KNeighborsClassifier_delete_missings,task15,0.882990,0.820618


 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [107]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

RF = RandomForestClassifier()
GBC = GradientBoostingClassifier()

acc_RF = cross_val_score(RF, X_new_df_MOD, y, scoring='accuracy').mean()
f1_RF = cross_val_score(RF, X_new_df_MOD, y, scoring='f1').mean()

acc_GB = cross_val_score(GBC, X_new_df_MOD, y, scoring='accuracy').mean()
f1_GB = cross_val_score(GBC, X_new_df_MOD, y, scoring='f1').mean()

results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

In [108]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.864100,0.760718
1,LogisticRegression,task13,0.904800,0.850846
2,KNeighborsClassifier,task13,0.886996,0.824782
3,LinearSVC,task13,0.906322,0.852914
12,Best_Model,task17,0.915395,0.866386
4,LogisticRegression_impute,task14,0.904659,0.850477
5,KNeighborsClassifier_impute,task14,0.887164,0.824966
6,LinearSVC_impute,task14,0.905142,0.850825
7,LogisticRegression_delete_missings,task15,0.901152,0.846845
8,KNeighborsClassifier_delete_missings,task15,0.882990,0.820618


17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [ ]:
from sklearn.compose import make_column_selector
from sklearn.model_selection import cross_validate

clf = [LinearSVC(), LogisticRegression(), RandomForestClassifier(), GradientBoostingClassifier()]
SIm = SimpleImputer(strategy='most_frequent', missing_values='?')
OHEn = OneHotEncoder(handle_unknown='ignore')

transf = Pipeline([('imputer', SIm), ('encoder', OHEn)])

CT_MMS = ColumnTransformer([("num", MinMaxScaler(), make_column_selector(dtype_exclude='category')),("categ", transf, make_column_selector(dtype_exclude='int64'))])
CT_SS = ColumnTransformer([("num", StandardScaler(), make_column_selector(dtype_exclude='category')),("categ", transf, make_column_selector(dtype_exclude='int64'))])
CT_MMS_HE = ColumnTransformer([("num", MinMaxScaler(), make_column_selector(dtype_exclude='category')),("categ", OHEn , make_column_selector(dtype_exclude='int64'))])
CT_SS_HE = ColumnTransformer([("num", StandardScaler(), make_column_selector(dtype_exclude='category')),("categ", OHEn, make_column_selector(dtype_exclude='int64'))])
P = Pipeline([('proc', CT_MMS),('clfs', GradientBoostingClassifier())])

params = {
    'clfs': clf,
    'proc': [CT_MMS,CT_SS,CT_MMS_HE,CT_SS_HE]}

GS = GridSearchCV(P, params, scoring='accuracy')
CV = cross_validate(GS, X_new_df_MOD, y, scoring=('f1', 'accuracy'))
GS.fit(X_new_df_MOD, y)
print(GS.best_score_)
print(GS.best_estimator_)
GS.best_params_

Подбирая наилучшую модель, компьютер считал более часа, но все продолжал считать, поэтому было решено прервать выполнение и сделать вывод по предыдущим заданиям.
Исходя выше из полученных значений, можно заметить что лучшим классификатором является GradientBoostingClassifier с f1 = 0.915395 и accuracy = 0.866386   

In [109]:
f1_best = 0.915395
acc_best = 0.866386
results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

In [110]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.864100,0.760718
1,LogisticRegression,task13,0.904800,0.850846
2,KNeighborsClassifier,task13,0.886996,0.824782
3,LinearSVC,task13,0.906322,0.852914
12,Best_Model,task17,0.915395,0.866386
4,LogisticRegression_impute,task14,0.904659,0.850477
5,KNeighborsClassifier_impute,task14,0.887164,0.824966
6,LinearSVC_impute,task14,0.905142,0.850825
7,LogisticRegression_delete_missings,task15,0.901152,0.846845
8,KNeighborsClassifier_delete_missings,task15,0.882990,0.820618
